In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os
import json
import glob

### Benchmark analysis

In [ ]:
output_path = "../output"
datasets = [f"MATH_TEST{elem}" for elem in [210, 245, 630]]
response_llms = [f"{llm}_{prompt}"
                 for llm in ["llama3-8b_vllm", "qwen2-7b_vllm", "llama3-70b_vllm", "qwen2-72b_vllm"]
                 for prompt in ["assistant", "math"]]
checker_llm = ["llama3-70b", "qwen2-72b"]

In [ ]:
out_dict = {}

# out should be df with columns set to datasets name, 
for checker in checker_llm:
    out_dict[checker] = []
    for dataset in datasets:
        dataset_eval = {}
        for response_llm in response_llms:
            file_path = os.path.join(output_path, f"{dataset}_{response_llm}")
            # df = pd.read_csv(glob.glob(os.path.join(file_path, f"report_{checker}_*.csv"))[0])
            json_reports = glob.glob(os.path.join(file_path, f"report_{checker}_*.json"))
            assert len(json_reports) >= 1
            json_report = json_reports[0]
            with open(json_report, "r") as f:
                report = json.load(f)
            dataset_eval[response_llm] = round(report["score"] * 100, 2)
        out_dict[checker].append(dataset_eval)
    out_dict[checker] = pd.DataFrame(out_dict[checker], index=datasets)

In [ ]:
out_dict["llama3-70b"]

In [ ]:
out_dict["qwen2-72b"]

### Insert answers to report csv

In [ ]:
for checker in checker_llm:
    for dataset in datasets:
        data = pd.read_csv(f"../data/{dataset}.csv")
        for response_llm in response_llms:
            file_path = os.path.join(output_path, f"{dataset}_{response_llm}")
            df = pd.read_csv(glob.glob(os.path.join(file_path, f"report_{checker}_*.csv"))[0])
            answers = []
            for i in df.index:
                answers.append(data.iloc[i]['Answer'])
            df["Answer"] = answers
            df.to_csv(f"{dataset}_{response_llm}_[{checker}].csv")
print("Done")